In [1]:
factor = 1.0
exactDate_mode = 'added_u'

# Feature Matrix Generation

In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), Swissbib's goldstandard data has been processed to form records of pairs of duplicate and pairs of unique records. These records are the starting point for the final feature matrix generation and that is the reason, why the DataFrame was called feature base. As described in [[JudACaps](./A_References.ipynb#judacaps)], the next step will be an attribute-wise comparison of each attribute pair of each record in the original feature base. This comparison will generate similarity values for each attribute pair. It will halve the number of attributes replacing each attribute pair with one value expressing their degree of similarity. The goal of this chapter is a DataFrame with the full and final feature attributes. The values of these feature attributes will be used for training and performance testing of the machine learning models in the chapters to follow.

This chapter introduces similarity metrics for string comparisons. The metrics to be used for calculating its similarity will be decided for each attribute pair of the DataFrame built in the previous chapters.

## Table of Contents

- [Data Takeover](#Data-Takeover)
- [Object Distance and Similarity](#Object-Distance-and-Similarity)
    - [Mathematical Definitions](#Mathematical-Definitions)
    - [Library TextDistance](#Library-TextDistance)
- [Similarity Metrics on Attribute Level](#Similarity-Metrics-on-Attribute-Level)
    - [Table of Contents of Attribute Similarities](#Table-of-Contents-of-Attribute-Similarities)
- [DataFrame with Attributes and Similarity Features](#DataFrame-with-Attributes-and-Similarity-Features)
- [Summary](#Summary)
    - [Full Feature Matrix with Target Vector Handover](#Full-Feature-Matrix-with-Target-Vector-Handover)

## Data Takeover

Swissbib's raw data of the goldstandard has been processed in chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb). As the first step of this chapter, this data is loaded for further processing to the feature matrix and target vector for the subsequent machine learning model chapters.

In [2]:
import os
import pandas as pd
import pickle as pk
import bz2
import _pickle as cPickle

path_goldstandard = './daten_goldstandard'

# Restore metadata so far
with open(os.path.join(path_goldstandard, 'columns_metadata.pkl'), 'rb') as handle:
    columns_metadata_dict = pk.load(handle)

# Restore DataFrame with features from compressed pickle file
with bz2.BZ2File((os.path.join(
    path_goldstandard, 'feature_base_df.pkl')), 'rb') as file:
    df_feature_base = cPickle.load(file)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_feature_base.columns)

df_feature_base.head()

,035liste_x,035liste_y,century_x,century_y,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,coordinate_x,coordinate_y,corporate_110_x,corporate_110_y,corporate_710_x,corporate_710_y,corporate_full_x,corporate_full_y,decade_x,decade_y,docid_x,docid_y,doi_x,doi_y,duplicates,edition_x,edition_y,exactDate_x,exactDate_y,format_postfix_x,format_postfix_y,format_prefix_x,format_prefix_y,isbn_x,isbn_y,ismn_x,ismn_y,masters_docid,musicid_x,musicid_y,pages_x,pages_y,part_x,part_y,person_100_x,person_100_y,person_245c_x,person_245c_y,person_700_x,person_700_y,pubinit_x,pubinit_y,pubword_x,pubword_y,pubyear_x,pubyear_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,ttlpart_x,ttlpart_y,volumes_x,volumes_y
0,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem englischen übersetzt von...,"grawechristian, graweursula","grawechristian, graweursula",reclam jun.,reclam jun.,[Reclam jun.],[Reclam jun.],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600,600
1,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem engl. übers. von ursula ...,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman",emma,,,"{'245': ['Emma', 'Roman']}",{'245': ['Emma']},600,600
2,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (LIBIB)000315536]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,323173349,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600,600
3,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem englischen übersetzt von...,,"grawechristian, graweursula",reclam,reclam jun.,[Reclam],[Reclam jun.],2009,2009,,,emma,"emma, roman",,,{'245': ['Emma']},"{'245': ['Emma', 'Roman']}",600,600
4,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem engl. übers. von ursula ...,,,reclam,reclam,[Reclam],[Reclam],2009,2009,,,emma,emma,,,{'245': ['Emma']},{'245': ['Emma']},600,600


Now, the feature base can be reduced to the columns used for processing.

In [3]:
# The DataFrame of pairs with target information
df_feature_base = df_feature_base[columns_metadata_dict['columns_to_use']]

df_feature_base.sample(n=5)

,duplicates,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,corporate_full_x,corporate_full_y,doi_x,doi_y,edition_x,edition_y,exactDate_x,exactDate_y,format_prefix_x,format_prefix_y,format_postfix_x,format_postfix_y,isbn_x,isbn_y,ismn_x,ismn_y,musicid_x,musicid_y,part_x,part_y,person_100_x,person_100_y,person_700_x,person_700_y,person_245c_x,person_245c_y,pubinit_x,pubinit_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,volumes_x,volumes_y
296726,1,e0075850,n0462000,n0462000,n0462000,eidgenössische landestopographie,iedgenössische landstopographie,,,,,1926uuuu,1926uuuu,mp,mp,010300,010300,[],[],,,,,15 15 1926,15 15 1926,,,,,,,[servizio topografico federale],[servizio topografico federale],100000,100000,domodossola,dmodososlh,,,2,2
88834,0,,,,,,,,,,,19739999,1963uuuu,cr,bk,030500,020000,[0335-1793],[],,,,,,,,austenjane,,,,jane austen,libération,peninsula press,,,libération,emma,,,,127
19241,0,,,,,,,,,,,2005uuuu,2008uuuu,bk,bk,020000,020053,[0-521-82437-0],"[978-3-598-31502-2 (print), 978-3-11-097002-9]",,,,,,22 22,austenjane,mortzfeldpeter,,raabepaul,jane austen ; ed. by richard cronin ... [et al.],"mortzfeld, peter; raabe, paul",,de gruyter saur,,,emma,katalog der graphischen porträts in der herzog...,,,599,1 361
309236,1,,,,,,,,,,,1992uuuu,1992uuuu,bk,bk,020053,020053,[978-3-598-31500-8 (print)],[978-3-598-31500-8 (print)],,,,,20 20,20 20,mortzfeldpeter,morztfeltdpeter,raabepaul,raabtepaul,"mortzfeld, peter; raabe, paul","mortzfeld, peter; raabe, paul",de gruyter saur,de gruyterersaur,,,katalog der graphischen porträts in der herzog...,katalog der graphischen portrtäs in der herzog...,,,1 418,1 418
226989,0,,,,,,"reussenhock, drach, kloster rheinau",,10.3931/e-rara-60823,,,1764uuuu,1475uuuu,bk,bk,020000,020053,[],[],,,,,,,voltaire,iodocus,,,,[jodocus erfordensis],[s.n.],[peter drach],,,traité sur la tolérance,vocabularius iuris utriusque,,,191,208


In [4]:
print('Number of rows labelled as duplicates {:,d}'.format(len(df_feature_base[
    df_feature_base.duplicates==1])))
print('Number of rows labelled as uniques {:,d}'.format(len(df_feature_base[
    df_feature_base.duplicates==0])))
print('Total number of rows in DataFrame {:,d}'.format(df_feature_base.shape[0],
      'number of columns', df_feature_base.shape[1]))

Number of rows labelled as duplicates 67,158
Number of rows labelled as uniques 257,955
Total number of rows in DataFrame 325,113


In [5]:
print('Part of duplicates (1) and uniques (0) in units of [%]')
print(round(100*df_feature_base.duplicates.value_counts(normalize=True), 2))

Part of duplicates (1) and uniques (0) in units of [%]
0    79.34
1    20.66
Name: duplicates, dtype: float64


DataFrame feature base is the starting point used for the further processing in this chapter.

## Object Distance and Similarity

A mathematical idea of distance and similarity is needed for understanding object pair comparison. This section starts with a motivation for calculating similarities and afterwards gives a very basic definition of the two central terms, distance and similarity. The text of this section is a summary of [[Chri2012](./A_References.ipynb#chri2012)].

### Mathematical Definitions

The attributes to be used for pair comparison may contain values of poor quality. The quality originates in the way the data has been entered at the very source. Manual data entry may suffer from mistyping, automatically scanned data may suffer from insufficiencies of the scanned base material or the recognition algorithm in the optical character recognition (OCR) processing. The basic step of a deduplication process is to identify the probability of two strings of a pair to be a pair of duplicates. This is done by calculating a similarity value between the two strings compared, rather than using an exact comparison function. Based on this common similarity value for an attribute pair, their being duplicates can be decided.

The term similarity is strongly coupled to the term of distance of two values of an attribute. Mathematically, a distance can be explained with the help of a distance function. A _distance function_ or _distance metric_ $dist(o_i, o_j)$ between two points or data objects $o_i$ and $o_j$ must fulfill four requirements.

1. $dist(o_i, o_i)=0$, the distance from an object to itself is zero.
- $dist(o_i, o_j)\ge 0$, the distance between two objects is a non-negative number.
- $dist(o_i, o_j)=dist(o_j, o_i)$, the distance between two objects is symmetric.
- $dist(o_i, o_j)\le dist(o_i, o_k)+dist(o_k, o_j)$, the triangular inequality must hold. It states that the direct distance beween two objects is never larger than the combined distance when going through a third object.

A distance value expresses the dissimilarity $d$ of two objects [[HanK2012](./A_References.ipynb#hank2012)] and can therefore be converted into a similarity value $s$, calculating $s = \frac{1}{d}$, assuming $d\gt 0$. Alternatively, assuming the distance value is normalised $0\le d\le 1$, the similarity value can be calculated to $s = 1-d$. A _similarity function_ $sim(a_i, aj)$ between two attributes which can be strings, numbers, dates, geographic locations, text, XML documents, etc. fulfills the general requirements.

1. $sim(a_i, a_i)=1$, the result of comparing a value with itself is an exact similarity.
- $sim(a_i, a_j)=0$, the similarity of values that are completely different from each other is 0. What accounts for 'complete different' depends upon the type of data that are compared.
- $0\lt sim(a_i, a_j)\lt 1$, an approximate similarity between exact similarity and total dissimilarity is calculated if two attribute values are somewhat similar to each other. What accounts for 'somewhat different' depends upon the type of data that are compared.

The dissimilarity between two objects $o_i$ and $o_j$ can be computed based on the ratio of mismatches,
$$
d(o_i, o_j) = \frac{p-m}{p},
$$
where $m$ is the number of matching attributes and $p$ is the total number of attributes describing the objects [[HanK2012](./A_References.ipynb#hank2012)]. Thus the similarity between two objects can be computed as
$$
sim(o_i, o_j) = 1 - d(o_i, o_j) = \frac{m}{p}.
$$

For data deduplication, a comparison function needs to be tailored to the type of underlying data. Although there is a correspondence between a similarity function and the mathematical concept of a distance function, not all known and implemented similarity comparison functions used for string pair comparison fulfill the requirements of a distance function. Some similarity functions are not symmetric, others do not fulfill the triangular inequality. Decision taking on the best similarity function for a string pair, will be based on the effect, a similarity function has for the purpose needed. In the case of this capstone project, this purpose is its capability to contribute to the prediction whether a pair of records is a pair duplicates or a pair of uniques.

### Library TextDistance

An internet research on string distance calculation with Python has revealed libraries [[StSi](./A_References.ipynb#stsi)], [[TeDi](./A_References.ipynb#tedi)] and seperate code snippets for distinct algorithms. After trying the referenced libraries and a downloaded code snippet for a Smith Waterman similarity [[SmWa](./A_References.ipynb#smwa)], the text distance library [[TeDi](./A_References.ipynb#tedi)] has been decided to be the best decision for this capstone project. The decision is based on the github statistics of stars and the date of the latest pull requests, indicating its popularity and maintenance activity of the library. A look at the API of the library, reveals the Python library to be a complete implementation (compared to suggestions of similarity metrics in [[Chri2012](./A_References.ipynb#chri2012)]) and easy to use.

In [6]:
# Install textdistance Python library - if not done, yet.
! pip install textdistance

For using the library, see documentation in [[TeDi](./A_References.ipynb#tedi)]. For the purposes of this chapter, function $\texttt{.normalized}\_\texttt{similarity}()$ of an instantiated textdistance object will be used.

In [7]:
import textdistance as tedi

With the code line above, the library is imported for application in this chapter. In appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb) the effects of the similarity metrics of the library are compared for better understanding of their specific behaviour. This comparison for each attribute is the basis of deciding the best similarity metric available for an attribute pair.

## Similarity Metrics on Attribute Level

This section implements the decision for calculating the similarity metric for each attribute of the raw data based on appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The implementation is applied on a pair of attributes of two records, resulting in a new attribute, the similarity value, of the final feature matrix. A general function $\texttt{.build}\_\texttt{delta}\_\texttt{feature}()$ is provided by the code file [data_preparation_funcs.py](./data_preparation_funcs.py) for transforming two attributes into their feature attribute holding their similarity value.

In [8]:
import data_preparation_funcs as dpf

The two dictionaries of the following code cell will be filled by function $\texttt{.build}\_\texttt{delta}\_\texttt{feature}()$.

In [9]:
columns_metadata_dict['similarity_metrics'] = {}
columns_metadata_dict['features'] = []

### Table of Contents of Attribute Similarities

- [coordinate](#coordinate)
- [corporate](#corporate)
- [doi](#doi)
- [edition](#edition)
- [exactDate](#exactDate)
- [format](#format)
- [isbn](#isbn)
- [ismn](#ismn)
- [musicid](#musicid)
- [part](#part)
- [person](#person)
- [pubinit](#pubinit)
- [scale](#scale)
- [ttlfull](#ttlfull)
- [volumes](#volumes)

### coordinate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{coordinate}$ holds coordinates of maps. To decide whether a map covers the same geographical range, a metric will be chosen that compares the coordinate number digits from left to right. The more digits are found to be equal, the higher the similarity value is calculated. The comparison stops with the first digit pair that differs. This algorithm is satisfied by the LCS (Longest Common Substring comparison) algorithm and generates the wanted result, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb).

In [10]:
attribute = 'coordinate'

columns_metadata_dict['similarity_metrics'][attribute+'_E'] = tedi.LCSStr()
columns_metadata_dict['similarity_metrics'][attribute+'_N'] = tedi.LCSStr()

ne_values = ['_E', '_N']

for ne in ne_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+ne,
        columns_metadata_dict['similarity_metrics'][attribute+ne],
        columns_metadata_dict)

The length of attribute $\texttt{coordinate}$ is exactly eight digits. The distinct similarity values that may occur form a discrete set of values with a distance of $\frac{1}{8}$ between adjacent values.

In [11]:
uniques, uniques_len = {}, {}

for ne in ne_values :
    uniques[attribute+ne], uniques_len[attribute+ne] = dpf.determine_similarity_values(
        df_feature_base, attribute+ne)

coordinate_E values range [0.    0.125 0.25  0.375 0.5   0.625 0.875 1.   ]
coordinate_N values range [0.    0.375 0.5   0.625 0.75  0.875 1.   ]


Looking at some samples of the feature matrix reveals a good match to the expectations.

In [12]:
position = 3

for ne in ne_values :
    dpf.show_samples_interval(
        df_feature_base, attribute+ne,
        uniques[attribute+ne][uniques_len[attribute+ne]-position],
        uniques[attribute+ne][uniques_len[attribute+ne]-position+1]
    )

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
69543,0,0.875,e0080855,e0080851
55311,0,0.875,e0080855,e0080851
196658,0,0.625,e0055700,e0055009
869,1,0.875,e0080855,e0080851
96189,0,0.875,e0080851,e0080855


0.625 <= coordinate_E_delta <= 0.875


,duplicates,coordinate_N_delta,coordinate_N_x,coordinate_N_y
55255,0,0.750,n0460826,n0460833
69514,0,0.750,n0460826,n0460833
55260,0,0.750,n0460826,n0460833
55263,0,0.750,n0460826,n0460833
863,1,0.875,n0460821,n0460826


0.75 <= coordinate_N_delta <= 0.875


The samples above show the wanted similarity behaviour for value ranges greater than 0. The metric has the weakness, though, that empty coordinate values, e.g. for bibliographic units other than maps, have each been calculated to a similarity of 0. Some samples for duplicates in the training data are shown below.

In [13]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute+'_E', uniques[attribute+'_E'][0], uniques[attribute+'_E'][1], 10)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
323292,1,0.0,,
277739,1,0.0,,
273458,1,0.0,,
277675,1,0.0,,
280807,1,0.0,,
291771,1,0.0,,
303686,1,0.0,,
259862,1,0.0,,
294287,1,0.0,,
316661,1,0.0,,


0.0 <= coordinate_E_delta <= 0.125


This downside shall be avoided by marking pairs of missing coordinate values on both sides with a special negative value, which will point out to the models to be trained, the special case of missing information in a row. The implementation of this logic is done in function $\texttt{.mark}\_\texttt{missing}()$. The absolute value of this negative number is conrolled by a factor which is passed to the function as a parameter. The function handles explicitly two cases. The first one is missing information in both attributes (resulting in $-1*\texttt{factor}$) and the second one is missing information in only one of the two attributes (resulting in $-0.5*\texttt{factor}$).

In [14]:
for ne in ne_values :
    df_feature_base = dpf.mark_missing(df_feature_base, attribute+ne, factor)

### corporate

Attribute $\texttt{corporate}$ is a collection of corporate names. The Monge-Elkan metric compares string tokens pairwise [[Chri2012](./A_References.ipynb#chri2012)] while the LCS metric searches for the longest common substring. Assessing the differences of these two metrics with the help of their values distribution in chapter [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb), reveals a better distribution behaviour for LCS. Therefore, the LCS metric will be chosen for this attribute.

In [15]:
attribute = 'corporate_full'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#tedi.StrCmp95()
#tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [16]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

corporate_full values range [0.         0.01333333 0.01428571 0.01639344 0.01666667 0.01694915
 0.01818182 0.01960784 0.02       0.02040816 0.0212766  0.02173913
 0.02272727 0.02380952 0.025      0.02531646 0.02542373 0.02666667
 0.02702703 0.02727273 0.02857143 0.03       0.03030303 0.03125
 0.03225806 0.03278689 0.03333333 0.03389831 0.03448276 0.03571429
 0.03636364 0.03773585 0.03797468 0.03846154 0.03921569 0.04
 0.04081633 0.04166667 0.04237288 0.04255319 0.04285714 0.04347826
 0.04444444 0.04545455 0.046875   0.04761905 0.04878049 0.04918033
 0.05       0.05263158 0.05333333 0.05405405 0.05454545 0.05555556
 0.05660377 0.05714286 0.05882353 0.05932203 0.06       0.06060606
 0.06122449 0.0625     0.06329114 0.06363636 0.06382979 0.06451613
 0.06521739 0.06557377 0.06666667 0.06779661 0.06818182 0.06896552
 0.07       0.07142857 0.07272727 0.07317073 0.075      0.0754717
 0.07594937 0.07627119 0.07692308 0.078125   0.07843137 0.07894737
 0.08       0.08108108 0.08163265 0.08181818

Its $110$ part is sparsely filled and even its $710$ part comes along with a little more than $10\%$ of filling, only. The LCS metric generates a similarity of 1 for the cases where both strings of a pair are empty. Missing values on both sides may be an indicator for a pair of duplicates but due to the sparsely available information, it is a weak indicator. Therefore, the pairs with missing data on both sides of the pair, will be marked with the negative value.

In [17]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

Some sample cases are shown below for both $\texttt{corporate}$ features.

In [18]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute, 0.0, 1.0, 20
)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
267085,1,0.837838,schweizbundesamt für landestopografie,schweizbundesamt für landestoporafie
320452,1,1.000000,schweizerische normen-vereinigung,schweizerische normen-vereinigung
299936,1,1.000000,historischer verein des kantons solothurn,historischer verein des kantons solothurn
302573,1,1.000000,"interkantonale lehrmittelzentrale (rapperswil,...","interkantonale lehrmittelzentrale (rapperswil,..."
270850,1,0.909091,uniwersytet im. adama mickiewicza w poznaniu,uniwersytet im. adama mickiewicza w poznnbiu
284263,1,0.366667,"interkantonale lehrmittelzentrale (rapperswil,...",interkantonyale lehrmitetlzentrale (rapperswil...
321623,1,0.666667,interkantonale lehrmittelzentrale (luzern),interkantonael lehrmittelzentrale (luzern)
276513,1,0.521127,"interkantonale lehrmittelzentrale, staatlicher...","interkantonale lehrmittelzentrale, stoaatliche..."
282035,1,0.967213,"interkantonale lehrmittelzentrale (rapperswil,...",iunterkantonale lehrmittelzentrale (rapperswil...
302306,1,0.549020,"opernhaus (zürich)orchester, opernhaus (zürich...","opernhaus z(ürich)orchester, opernhaus (lürich..."


0.0 <= corporate_full_delta <= 1.0


In [19]:
position = uniques_len[attribute]//2 # Let's have a look in the middle range of the similarities.

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 20)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
300697,1,0.609756,trägerverein «600 jahre niklaus von flüe»,trägerverein «600 jahre nkilaus von flüe»
300752,1,0.609756,trägerverein «600 jahre niklaus von flüe»,trägerverein «600 jahre nkilaus von flüe»
300695,1,0.609756,trägerverein «600 jahre niklaus von flüe»,trägerverein «060 jahre niklaus von flüe»
299953,1,0.609756,historischer verein des kantons solothurn,historischer verein des kntons solothurn
300700,1,0.609756,trägerverein «600 jahre niklaus von flüe»,trägeverein «60h0 jahre niklaus von flüe»
318037,1,0.609091,eidgenössisches topographisches bureau eidgenö...,eidgenössisches topographisches bureau eidgenö...
300672,1,0.609756,trägerverein «600 jahre niklaus von flüe»,trägerverein «060 jahre niklaus von flüe»
318031,1,0.609091,eidgenössisches topographisches bureau eidgenö...,eidgenössisches topographisches bureau eidgenö...
263032,1,0.610000,schweizerische akademie der geisteswissenschaf...,schweizerische akademie der geisteswissenschaf...
299901,1,0.609756,historischer verein des kantons solothurn,historiscwher veein des kantons solothurn


0.6090909090909091 <= corporate_full_delta <= 0.61


### doi

Swissbib uses an explicit $\texttt{doi}$ attribute for its deduplication implementation. In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), the real doi identifier has been isolated with the help of a preprocessing function $\texttt{.reduce}\_\texttt{to}\_\texttt{doi}\_\texttt{element}()$, see [Data Analysis](./1_DataAnalysis.ipynb). Attribute $\texttt{doi}$ contains a single string value. The Identity metric will be used for comparing the string values of a pair in a row, calculating a similarity value of 1.0 or 0.0 for each pair. If one list is empty a value of 0 is returned.

In [20]:
attribute = 'doi'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base['doi_delta'].unique()

array([1., 0.])

Some sample cases are shown below for each category of $\texttt{doi}\_\texttt{delta}$.

In [21]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

doi values range [0. 1.]


,duplicates,doi_delta,doi_x,doi_y
147185,0,1.0,,
217254,0,1.0,,
229225,0,1.0,,
126708,0,1.0,,
210261,0,1.0,,
27261,0,1.0,,
66059,0,1.0,,
232549,0,1.0,,
209767,0,1.0,,
144555,0,1.0,,


doi_delta = 1.0


,duplicates,doi_delta,doi_x,doi_y
141192,0,0.0,,10.5169/seals-376810
74177,0,0.0,,10.5169/seals-376572
166795,0,0.0,10.1093/ndt/gft319,
208739,0,0.0,10.1093/ndt/gft319,
96127,0,0.0,,10.5169/seals-376925
194236,0,0.0,10.5451/unibas-006503313,
194477,0,0.0,10.5451/unibas-006503313,
208239,0,0.0,10.1093/cid/cir669,10.5169/seals-515321
207791,0,0.0,10.1093/cid/ciu795,
98308,0,0.0,,10.3931/e-rara-60823


doi_delta = 0.0


In [22]:
# Let's have a look at some non-empty doi elements
df_doi_with_element = df_feature_base[df_feature_base.doi_x.apply(lambda x : len(x) > 0)]

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_doi_with_element, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

,duplicates,doi_delta,doi_x,doi_y
290453,1,1.0,10.5169/seals-376510,10.5169/seals-376510
275146,1,1.0,10.3931/e-rara-61897,10.3931/e-rara-61897
290715,1,1.0,10.5169/seals-376605,10.5169/seals-376605
319247,1,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
294570,1,1.0,10.5169/seals-515332,10.5169/seals-515332
319214,1,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
318281,1,1.0,10.1055/b-002-26639,10.1055/b-002-26639
292278,1,1.0,10.5169/seals-377277,10.5169/seals-377277
294550,1,1.0,10.5169/seals-515332,10.5169/seals-515332
290130,1,1.0,10.5169/seals-376359,10.5169/seals-376359


doi_delta = 1.0


,duplicates,doi_delta,doi_x,doi_y
208625,0,0.0,10.1093/ndt/gft319,
255495,0,0.0,10.1055/b-005-143650,
207561,0,0.0,10.1093/cid/ciu795,
200549,0,0.0,10.1007/978-3-642-41698-9,
255349,0,0.0,10.1055/b-005-143650,
208800,0,0.0,10.1093/ndt/gft319,10.5169/seals-377028
200507,0,0.0,10.1007/978-3-642-41698-9,10.5169/seals-377332
208286,0,0.0,10.1093/cid/cir669,
194655,0,0.0,10.5451/unibas-006503313,
166777,0,0.0,10.1093/ndt/gft319,


doi_delta = 0.0


As can be seen above, a value of 1.0 is returned if both strings of a pair are empty. As the attribute filling of $\texttt{doi}$ is sparse, see chapter [Data Analysis](./1_DataAnalysis.ipynb), the $\texttt{doi}\_\texttt{delta}$ indicates strongly a pair of duplicates for most rows. To avoid such misleading identity indication, function $\texttt{.mark}\_\texttt{missing}()$ will be applied to the attribute.

In [23]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### edition

In its original form in Swissbib's raw data, the edition statement is a string value which may have several words. The modelling on this attribute has been tried with and without stripping letter characters from the string. The final decision for the best processing will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb). A Jaccard similarity is tried for this attribute.

In [24]:
attribute = 'edition'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [25]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

import numpy as np

edition_delta_uniques = np.sort(df_feature_base['edition_delta'].unique())
edition_delta_uniques_len = len(edition_delta_uniques)
print('edition values range', edition_delta_uniques[:30])

edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]
edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]


The comparison results in a wide number of distinct similarity values for the goldstandard data set. Below, some examples are shown.

In [26]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position],
    edition_delta_uniques[edition_delta_uniques_len-position+2], 10)

position = edition_delta_uniques_len//2

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
261863,1,1.0,,
291997,1,1.0,,
293412,1,1.0,,
264293,1,1.0,,
177617,0,1.0,,
224308,0,1.0,,
269082,1,1.0,,
295090,1,1.0,,
169005,0,1.0,,
113649,0,1.0,,


0.6 <= edition_delta <= 1.0


,duplicates,edition_delta,edition_x,edition_y
180852,0,0.0,6,
29137,0,0.0,5,
241831,0,0.0,,3
116456,0,0.0,,2
45296,0,0.0,,2
175107,0,0.0,6,
94438,0,0.0,,1899
77040,0,0.0,3,2
66760,0,0.0,,2
68962,0,0.0,2,


0.0 <= edition_delta <= 0.1428571428571428


,duplicates,edition_delta,edition_x,edition_y
161673,0,0.25,6,1869
173533,0,0.25,6,1869
132620,0,0.25,5,1905
184428,0,0.20,1863,12
174126,0,0.25,6,1869
41809,0,0.25,1791,7
134134,0,0.25,3,1943
184808,0,0.25,6,1926
248370,0,0.20,10425,4
4867,0,0.25,1,1998


0.19999999999999996 <= edition_delta <= 0.25


Again, for $\texttt{edition}\_\texttt{delta} = 1$, many empty values of the $\texttt{edition}$ attribute can be observed. These will be marked with the special negative value in the data with the goal to distinguish them from the matching attribute pairs.

In [27]:
df_feature_base = dpf.mark_missing(df_feature_base, 'edition', factor)

In [28]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
109896,0,1.0,4,4
294280,1,1.0,5,5
300267,1,1.0,2007,2007
269974,1,1.0,4,4
264209,1,1.0,8,8
283306,1,1.0,2,2
295873,1,1.0,2000,2000
289860,1,1.0,1,1
267337,1,1.0,1869,1869
300298,1,1.0,2007,2007


0.6 <= edition_delta <= 1.0


### exactDate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{exactDate}$ holds a year number stored in the first four digits. Letter 'u' is used as a placeholder for an unknown digit. The attribute may hold some month and day or a second year information in its second four digits, additionally.

The attribute will be kept as a string and will not be transformed to an integer. The feature attribute of the record pair to be compared will be calculated with a modified Hamming algorithm, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The resulting similarity will be stored in a new attribute $\texttt{exactDate}\_\texttt{delta}$ which will be taken for the model calculation.

As can be seen in chapter [Decision Tree Model](./6_DecisionTreeModel.ipynb), this attribute is important for prediction. Different ways of increasing the weight of the unknown status of a digit have been tried. The different ways can be seen in the implementations below. The algorithm decided for the final simulation will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb).

In [29]:
import string

def no_xor (x_side, y_side) :
    number = 0
    for i in range(len(x_side)) :
        if ((x_side[i] in string.ascii_lowercase) | (y_side[i] in string.ascii_lowercase)) & (x_side[i] != y_side[i]) :
            number = number + 1
    return number

print('Example comparison results in a value of', no_xor ('202a0aaa', '1920uuuu'))

Example comparison results in a value of 5


In [30]:
attribute = 'exactDate'

# Replace letter 'u' with letter 'a' for one of the two strings.
#  As an effect, the resulting Hamming similarity for a letter
#  instead of a numerical digit in either string will add with an amount 0.
df_feature_base[attribute+'_x'] = df_feature_base.exactDate_x.str.replace('u', 'a')

# Compute Hamming similarity for century string pair.
columns_metadata_dict['similarity_metrics'][attribute] = tedi.Hamming()

unknown_share = 16

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

if exactDate_mode == 'added_u':
    # Add amount of 1/16 to Hamming similarity for every letter digit.
    #  But only maximum number of letter digits in both strings of a pair.
    df_feature_base[attribute+'_delta'] = df_feature_base[[
        attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
        lambda x : x[attribute+'_delta'] + 
        max(x[attribute+'_x'].count('a'), x[attribute+'_y'].count('u'))/unknown_share, axis=1
    )
elif exactDate_mode == 'xor':
    # Add amount of 1/16 to Hamming similarity for every letter digit.
    #  But only number of position-wise xor-ed letter digits in the two strings of a pair.
    df_feature_base[attribute+'_delta'] = df_feature_base[[
        attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
        lambda x : x[attribute+'_delta'] + 
        no_xor(x[attribute+'_x'], x[attribute+'_y'])/unknown_share, axis=1
    )

In [31]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']].sample(n=10)

,exactDate_x,exactDate_y,exactDate_delta
34509,19959999,1939uuuu,0.500
295384,1896aaaa,1896uuuu,0.750
15570,1976aaaa,2016uuuu,0.375
55287,1963aaaa,2013uuuu,0.375
283524,2004aaaa,2004uuuu,0.750
62804,aaaaaaaa,1997uuuu,0.500
59203,aaaaaaaa,2005uuuu,0.500
174752,20159999,1980uuuu,0.250
173701,20159999,1999uuuu,0.250
259243,2016aaaa,1869uuuu,0.250


All resulting values of equal strings are equal to 1.

In [32]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']][
    df_feature_base.exactDate_x == df_feature_base.exactDate_y
].sort_values('exactDate_delta', ascending=False).head()

,exactDate_x,exactDate_y,exactDate_delta
159,20022000,20022000,1.0
303999,20172016,20172016,1.0
304015,20172016,20172016,1.0
304014,20172016,20172016,1.0
304013,20172016,20172016,1.0


A discrete set of different similarity values can be found in the attribute deltas. Some sample records are shown below.

In [33]:
exactDate_deltas = np.sort(df_feature_base.exactDate_delta.unique())
exactDate_deltas

array([0.    , 0.125 , 0.25  , 0.3125, 0.375 , 0.4375, 0.5   , 0.5625,
       0.625 , 0.6875, 0.75  , 0.875 , 1.    ])

In [34]:
sample_size = 5

for i in exactDate_deltas :
    dpf.show_samples_distinct(df_feature_base, 'exactDate', i, sample_size)
    print(f'exactDate_delta = {i}')

,duplicates,exactDate_delta,exactDate_x,exactDate_y
201187,0,0.0,20151475,19669999
125403,0,0.0,19819999,20162007
168692,0,0.0,19791999,20162007
186737,0,0.0,20151475,19969999
124123,0,0.0,19829999,20012002


exactDate_delta = 0.0


,duplicates,exactDate_delta,exactDate_x,exactDate_y
186816,0,0.125,20151475,19001950
174631,0,0.125,20159999,17931797
240360,0,0.125,19911794,20151475
173679,0,0.125,20159999,19001950
42298,0,0.125,19561791,20151475


exactDate_delta = 0.125


,duplicates,exactDate_delta,exactDate_x,exactDate_y
74829,0,0.25,2000aaaa,1943uuuu
140588,0,0.25,2005aaaa,1981uuuu
97767,0,0.25,2002aaaa,19949999
204924,0,0.25,2011aaaa,1473uuuu
174989,0,0.25,20159999,1988uuuu


...

exactDate_delta = 0.75


,duplicates,exactDate_delta,exactDate_x,exactDate_y
313211,1,0.875,18801890,08801890
281044,1,0.875,19969999,19919999
314667,1,0.875,19911990,49911990
263063,1,0.875,19aa9999,19uu9999
299957,1,0.875,19289999,09289999


exactDate_delta = 0.875


,duplicates,exactDate_delta,exactDate_x,exactDate_y
312685,1,1.0,19972002,19972002
314186,1,1.0,19201930,19201930
260225,1,1.0,19969999,19969999
302453,1,1.0,19801983,19801983
294314,1,1.0,19631970,19631970


exactDate_delta = 1.0


### format

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{format}$ has been split up into two new attributes $\texttt{format}\_\texttt{prefix}$ and $\texttt{format}\_\texttt{postfix}$ which will be compared by a different similarity metrics.

- As the quality of $\texttt{format}\_\texttt{prefix}$ is expected to be high, an identity comparison should be sufficient.
- Due to the observed structure of $\texttt{format}\_\texttt{postfix}$, a q-gram based comparison will be chosen.

In [35]:
attribute = 'format'

columns_metadata_dict['similarity_metrics'][attribute+'_prefix'] = tedi.Identity()
columns_metadata_dict['similarity_metrics'][attribute+'_postfix'] = tedi.Jaccard(qval=2)

pfix_values = ['_prefix', '_postfix']

for pf in pfix_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pf,
        columns_metadata_dict['similarity_metrics'][attribute+pf],
        columns_metadata_dict)

In [36]:
for i in df_feature_base.format_prefix_delta[
    df_feature_base.format_prefix_x != df_feature_base.format_prefix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_prefix', i)
    print(f'format_prefix_delta = {i}')

,duplicates,format_prefix_delta,format_prefix_x,format_prefix_y
35453,0,0.0,bk,vm
10834,0,0.0,vm,cr
242871,0,0.0,bk,vm
6090,0,0.0,vm,cr
74709,0,0.0,vm,bk


format_prefix_delta = 0.0


In [37]:
for i in df_feature_base.format_postfix_delta[
    df_feature_base.format_postfix_x != df_feature_base.format_postfix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_postfix', i)
    print(f'format_postfix_delta = {i}')

,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
146221,0,0.428571,020000,010200
218014,0,0.428571,030000,010300
114166,0,0.428571,020000,010200
139768,0,0.428571,010300,010000
114269,0,0.428571,020000,010000


format_postfix_delta = 0.4285714285714286


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
178274,0,0.111111,010300,020000
14133,0,0.111111,010300,020000
137630,0,0.111111,020000,010053
242108,0,0.111111,020000,010100
245069,0,0.111111,020053,030653


format_postfix_delta = 0.11111111111111116


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
173293,0,0.25,020000,020347
20576,0,0.25,010347,010200
240313,0,0.25,020000,020353
205339,0,0.25,020053,010253
87925,0,0.25,010100,010253


...

format_postfix_delta = 0.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
11900,0,1.0,020000,020000
319768,1,1.0,020053,020053
283213,1,1.0,020000,020000
300694,1,1.0,020400,020400
60043,0,1.0,020000,020000


format_postfix_delta = 1.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
275681,1,0.666667,010353,010335
323150,1,0.666667,020300,200300
268871,1,0.666667,010200,100200
298795,1,0.666667,020053,200053
297776,1,0.666667,020000,200000


format_postfix_delta = 0.6666666666666666


### isbn

Swissbib uses each string element of the $\texttt{isbn}$ list separately for comparing with each string element of its comparison $\texttt{isbn}$ list. If two bibliographic units hold at least one element in common, this is interpreted as a strong indicator for duplicates [[WiCo2001](./A_References.ipynb#wico2001)].

This hard logic is used in a modified way in the context of this capstone project. A special comparison function $\texttt{.build}\_\texttt{delta}\_\texttt{isbn}()$ has been implemented that compares each list element of the left-hand side with each list element of the right-hand side of a pair. According to Swissbib's implementation, the Identity metric is used for string comparison, calculating a similarity value of 1.0 or 0.0 for each list element pair. For normalisation reasons, the sum of similarity values is divided by the number of elements of the smaller list. If both lists are empty a value of 1.0 is returned. If only one list is empty a value of 0.0 is returned.

In [38]:
attribute = 'isbn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base[attribute+'_delta'].unique()

array([1. , 0. , 0.5])

Some sample cases are shown below for each category of $\texttt{isbn_delta}$.

In [39]:
for isbn_delta_value in df_feature_base['isbn_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['isbn_delta']==isbn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'isbn', isbn_delta_value, number_of_max_samples)
    print(f'isbn_delta = {isbn_delta_value}')

,duplicates,isbn_delta,isbn_x,isbn_y
280410,1,1.0,[3-906721-58-2],[3-906721-58-2]
232282,0,1.0,[],[]
178557,0,1.0,[],[]
260428,1,1.0,[3-442-33001-7],[3-442-33001-7]
166361,0,1.0,[],[]
305719,1,1.0,[],[]
317163,1,1.0,[],[]
142221,0,1.0,[],[]
42191,0,1.0,[],[]
300464,1,1.0,[3-7618-0609-4],[3-7618-0609-4]


isbn_delta = 1.0


,duplicates,isbn_delta,isbn_x,isbn_y
181187,0,0.0,"[978-3-642-41697-2, 3-642-41697-7]",[978-3-7815-1513-0]
158806,0,0.0,[978-3-643-12370-1],[]
115422,0,0.0,[0252-9955],[]
72404,0,0.0,[978-3-7255-6535-1],[]
134117,0,0.0,[3-13-127283-X],[]
235785,0,0.0,[978-2-226-31734-6],[]
259365,0,0.0,"[978-3-290-20138-8, 3-290-20138-4]",[]
196358,0,0.0,[978-3-495-48796-9],[]
3147,0,0.0,[3-495-47879-5],[]
106234,0,0.0,[],[978-3-526-52644-5]


isbn_delta = 0.0


,duplicates,isbn_delta,isbn_x,isbn_y
1195,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1199,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1202,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1205,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1201,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1210,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"


isbn_delta = 0.5


For attribute $\texttt{isbn}$, the special marking of missing values is omitted.

### ismn

This attribute will be processed with the identity similarity metric. The reasoning for this decision is the same as for similar attributes above. 

In [40]:
attribute = 'ismn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()
#tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [41]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

ismn values range [0. 1.]


In [42]:
for ismn_delta_value in df_feature_base[attribute+'_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base[attribute+'_delta']==ismn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'ismn', ismn_delta_value, number_of_max_samples)
    print(f'ismn_delta = {ismn_delta_value}')

,duplicates,ismn_delta,ismn_x,ismn_y
246137,0,1.0,,
209235,0,1.0,,
289958,1,1.0,,
9843,0,1.0,,
86141,0,1.0,,
190793,0,1.0,,
146023,0,1.0,,
83414,0,1.0,,
86280,0,1.0,,
180874,0,1.0,,


ismn_delta = 1.0


,duplicates,ismn_delta,ismn_x,ismn_y
249705,0,0.0,9790006450510,
27165,0,0.0,,9790006450510
78310,0,0.0,,m008060205
99221,0,0.0,m006546749,
135140,0,0.0,m006450510,
250059,0,0.0,m200205343,
256382,0,0.0,"m006546756 (kritischer bericht, leinen)",
58435,0,0.0,,m700241001
20341,0,0.0,m006450510,
115149,0,0.0,"m006546756 (kritischer bericht, leinen)",


ismn_delta = 0.0


As can be seen in the previous chapters, attribute $\texttt{ismn}$ is filled sparsely. A lot of missing values calculate to a value of 1.0 in the chosen similarity metrics. To mark these cases specifically, they will be transformed to a negative value.

In [43]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### musicid

Chapter [Data Analysis](./1_DataAnalysis.ipynb) shows that attribute $\texttt{musicid}$ is an identifyer for a music record. A Jaccard metric has been tested on this attribute, resulting in a distribution of many high similarity values on uniques. Comparing this result with the LCS metric, the latter has been decided.

In [44]:
attribute = 'musicid'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [45]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

musicid values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.375      0.4        0.42857143 0.5
 0.55555556 0.57142857 0.6        0.625      0.66666667 0.71428571
 0.75       0.77777778 0.8        0.83333333 0.85714286 0.875
 0.88888889 1.        ]


In [46]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//2

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)

,duplicates,musicid_delta,musicid_x,musicid_y
260955,1,1.0,502023,502023
298715,1,1.0,172,172
313078,1,1.0,422,422
322664,1,1.0,99036,99036
270446,1,1.0,8962,8962
301781,1,1.0,95088,95088
260673,1,1.0,92633,92633
271568,1,1.0,4355,4355
268800,1,1.0,542,542
268797,1,1.0,542,542


0.8888888888888888 <= musicid_delta <= 1.0


,duplicates,musicid_delta,musicid_x,musicid_y
148792,0,0.0,433,
71929,0,0.0,,
250531,0,0.0,3714,
30028,0,0.0,,
263683,1,0.0,,
90980,0,0.0,,
232752,0,0.0,,
63728,0,0.0,5944,
317302,1,0.0,,
170844,0,0.0,415,


0.0 <= musicid_delta <= 0.1428571428571429


,duplicates,musicid_delta,musicid_x,musicid_y
289127,1,0.600000,5944,594d4
284725,1,0.600000,33001,3301
86017,0,0.600000,99036,99064
322679,1,0.571429,501326,50y1326
304760,1,0.571429,4357412,435712
316229,1,0.600000,10003,1003
313495,1,0.600000,4553,455g3
316231,1,0.600000,10003,1003
274563,1,0.600000,10425,1045
279743,1,0.600000,8571,8k571


0.5714285714285714 <= musicid_delta <= 0.6


In [47]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1], attribute,
    uniques[attribute][0],
    uniques[attribute][uniques_len[attribute]-1], 20)

,duplicates,musicid_delta,musicid_x,musicid_y
320109,1,0.000000,,
324578,1,1.000000,422,422
319298,1,0.000000,,
316844,1,0.000000,,
265298,1,0.000000,,
271983,1,0.000000,,
317912,1,0.000000,,
282974,1,0.000000,,
313639,1,1.000000,4553,4553
321202,1,0.000000,,


0.0 <= musicid_delta <= 1.0


The attribute is filled with a degree of below $10\%$. The chosen metric for it results in a similarity value of 1.0 for empty value pairs. This effect can be adjusted with function $\texttt{.mark}\_\texttt{missing}()$ as above. 

In [48]:
df_feature_base = dpf.mark_missing(df_feature_base, 'musicid', factor)

### part

Analogous to attribute $\texttt{edition}$ described above, the string value of this attribute can be stripped to pure number digits. Both ways, with and without letter stripping have been tried for modelling. The final decision for the best processing will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb). Three different metrics have been tried for attribute $\texttt{part}$. Finally, metric StringCompare95 will be used.

In [49]:
attribute = 'part'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
#tedi.Jaro()
#tedi.Hamming()
#tedi.LCSStr()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [50]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

part values range [0.         0.27407407 0.28240741 0.29202279 0.30740741 0.31318681
 0.31481481 0.32407407 0.32898551 0.33333333 0.33597884 0.33838384
 0.33921569 0.34444444 0.3452381  0.35128205 0.35185185 0.35897436
 0.36111111 0.36231884 0.36666667 0.37037037 0.37179487 0.37254902
 0.37301587 0.37407407 0.375      0.37777778 0.38333333 0.38461538
 0.38568376 0.38888889 0.38927739 0.39070048 0.39215686 0.39393939
 0.3952381  0.3960114  0.39646465 0.39722222 0.39814815 0.4
 0.4031746  0.4037037  0.40604575 0.40740741 0.40842491 0.40855763
 0.41025641 0.41111111 0.41125541 0.41176471 0.41203704 0.41282051
 0.41388889 0.41449275 0.41452991 0.41507937 0.41666667 0.4178744
 0.41798942 0.41851852 0.41919192 0.42083333 0.42222222 0.42261905
 0.42390289 0.4241453  0.42564103 0.42592593 0.42777778 0.42810458
 0.42857143 0.42948718 0.43030303 0.43055556 0.43115942 0.43174603
 0.43236715 0.43333333 0.43518519 0.43557423 0.43589744 0.43627451
 0.43650794 0.43703704 0.4375     0.43813131 0.43888

In [51]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//7

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

,duplicates,part_delta,part_x,part_y
205122,0,1.0,,
160270,0,1.0,,
313078,1,1.0,,
134623,0,1.0,,
168554,0,1.0,,
254547,0,1.0,,
94960,0,1.0,,
302929,1,1.0,,
260313,1,1.0,7476,7476
299930,1,1.0,,


0.9215686274509803 <= part_delta <= 1.0


,duplicates,part_delta,part_x,part_y
189287,0,0.0,208,
153109,0,0.0,,23 23 1939
10726,0,0.0,,265 1986
172984,0,0.0,208,
250970,0,0.0,,23 23 1899 23
121698,0,0.0,,23 23 1889 23
6621,0,0.0,,293 2014
253632,0,0.0,2017 313 74,
119817,0,0.0,,21
163171,0,0.0,,291 2012


0.0 <= part_delta <= 0.28240740740740744


,duplicates,part_delta,part_x,part_y
238537,0,0.714286,2,23 1902
216177,0,0.714286,552 552,2
197168,0,0.714286,23 1862,2
197338,0,0.714286,23 1862,2
117706,0,0.714286,552 552,2
58882,0,0.714286,2,241 319
132414,0,0.714286,4,241 319
214535,0,0.714286,912 912,2
214507,0,0.714286,912 912,2
121147,0,0.714286,2,241 319


0.7142857142857143 <= part_delta <= 0.7146464646464646


,duplicates,part_delta,part_x,part_y
183896,0,0.716667,26 2000 26 2000,279 2000
183898,0,0.716667,26 2000 26 2000,281 2002
197572,0,0.717338,23 1862,23 23 1870 23
257465,0,0.717338,23 1862,23 23 1870 23
257495,0,0.717338,23 1862,23 23 1889 23
197586,0,0.717338,23 1862,23 23 1988 23
197585,0,0.717338,23 1862,23 23 1946 23
37481,0,0.716667,5 19,15 15 1926
197596,0,0.717338,23 1862,23 23 1870 23
257479,0,0.717338,23 1862,23 23 1988 23


0.7159544159544161 <= part_delta <= 0.7173382173382173


In this attribute, too, moving pairs of empty values to negative values will result in a clearer distinction between pairs of uniques and duplicates, as will be seen in the graphical comparison of capter [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb).

In [52]:
df_feature_base = dpf.mark_missing(df_feature_base, 'part', factor)

### person

As a result of chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{person}$ has been split into three specific attributes. Attribute $\texttt{person}\_{100}$ and $\texttt{person}\_{700}$ hold strongly standardised string values. For comparing pure strings, a Levenshtein metric is recommended [[Chri2012](./A_References.ipynb#chri2012)]. Unfortunately, this metric shows a very long calculation time on the data of the capstone project. Comparing the similarity values of the Levenshtein metric with the similarity values of other metrics in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), similarity metric StrCmp95 has been decided to use.

In [53]:
attribute = 'person'

columns_metadata_dict['similarity_metrics'][attribute+'_100'] = tedi.StrCmp95()
columns_metadata_dict['similarity_metrics'][attribute+'_700'] = tedi.StrCmp95()
#tedi.Levenshtein()

pe_values = ['_100', '_700']

for pe in pe_values :
    print('Calculating person'+pe)
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pe,
        columns_metadata_dict['similarity_metrics'][attribute+pe],
        columns_metadata_dict)

Calculating person_100


Calculating person_700


In [54]:
pe = '_100'

uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_100 values range [0.         0.31944444 0.32777778 ... 0.99090909 0.99130435 1.        ]


In [55]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_100_delta,person_100_x,person_100_y
282735,1,1.0,,
182524,0,1.0,,
123043,0,1.0,,
297060,1,1.0,,
197655,0,1.0,,
261762,1,1.0,austenjane,austenjane
317908,1,1.0,,
70083,0,1.0,,
268548,1,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
301673,1,1.0,,


0.9913043478260869 <= person_100_delta <= 1.0


,duplicates,person_100_delta,person_100_x,person_100_y
168678,0,0.0,,mortzfeldpeter
242552,0,0.0,voltaire,
213829,0,0.0,,austenjane
224858,0,0.0,rosoffmeg,
226284,0,0.0,,austenjane
127429,0,0.0,mozartwolfgang amadeus,
116651,0,0.0,mozartwolfgang amadeus,
200962,0,0.0,petrarcafrancesco,
176902,0,0.0,,petrarcafrancesco
92060,0,0.0,,beethovenludwig van


0.0 <= person_100_delta <= 0.3277777777777777


For comparing person names, like in attribute $\texttt{person}\_{245c}$, a Jaro metric will be tested [[Chri2012](./A_References.ipynb#chri2012)].

In [56]:
pe = '_245c'

columns_metadata_dict['similarity_metrics'][attribute+pe] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute+pe,
    columns_metadata_dict['similarity_metrics'][attribute+pe],
    columns_metadata_dict)

In [57]:
uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_245c values range [0.         0.24632035 0.25462963 ... 0.99881797 0.99882214 1.        ]


In [58]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_245c_delta,person_245c_x,person_245c_y
272255,1,1.0,sigrid kessler ... [et al.],sigrid kessler ... [et al.]
276312,1,1.0,"sigrid keller, caty laubscher, helen wallimann...","sigrid keller, caty laubscher, helen wallimann..."
293816,1,1.0,voltaire ; [sous la dir. de diego venturino],voltaire ; [sous la dir. de diego venturino]
322333,1,1.0,liane moriarty ; trad. de l'anglais (australie...,liane moriarty ; trad. de l'anglais (australie...
261328,1,1.0,von w.a. mozart ; dichtung nach ludwig gieseck...,von w.a. mozart ; dichtung nach ludwig gieseck...
322999,1,1.0,von beatrice käser,von beatrice käser
283986,1,1.0,w. a. mozart ; libretto: emanuel schikaneder ;...,w. a. mozart ; libretto: emanuel schikaneder ;...
306519,1,1.0,"mortzfeld, peter; raabe, paul","mortzfeld, peter; raabe, paul"
274026,1,1.0,von w.a. mozart ; dichtung nach ludwig gieseck...,von w.a. mozart ; dichtung nach ludwig gieseck...
287509,1,1.0,w. a. mozart ; nach dem in der deutschen staat...,w. a. mozart ; nach dem in der deutschen staat...


0.9988221436984688 <= person_245c_delta <= 1.0


,duplicates,person_245c_delta,person_245c_x,person_245c_y
90020,0,0.0,,jane austen
68259,0,0.0,,[walter bührer]
34199,0,0.0,,von w.a. mozart ; dichtung nach ludwig gieseck...
153812,0,0.0,w. a. mozart ; texte de schikaneder,
213294,0,0.0,,wolgang amadeus mozart ; dieser opernführer wu...
252441,0,0.0,reto u. schneider [und 7 andere],
56746,0,0.0,beatrice käser,
161519,0,0.0,von emanuel schikaneder ; [musik von] wolfgang...,
30102,0,0.0,andreas flury,
85557,0,0.0,hrsg. von michaela gläser-zikuda und tina hascher,


0.0 <= person_245c_delta <= 0.25462962962962954


The similarities of all three $\texttt{person}$ attributes are affected by empty values. These will be handled the same way as the attributes above.

In [59]:
pe_values = ['_100', '_245c', '_700']

for pe in pe_values :
    df_feature_base = dpf.mark_missing(df_feature_base, 'person'+pe, factor)

### pubinit

This attribute holds publisher strings that have a similar representation as attribute $\texttt{person}$. A Jaro metric will be used.

In [60]:
attribute = 'pubinit'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [61]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

pubinit values range [0.         0.25132275 0.25303644 ... 0.9957265  0.99578059 1.        ]


In [62]:
position = uniques_len[attribute]//3

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-1],
    uniques[attribute][uniques_len[attribute]-position], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-5],
    uniques[attribute][uniques_len[attribute]-1], 10)

,duplicates,pubinit_delta,pubinit_x,pubinit_y
48171,0,0.594822,"klett und balmer, universitätsverlag, universi...",cambridge university press
201184,0,0.594830,"adv, biblioteca cantonale di lugano","in melanges, ed. van den heuvel. paris, gallimard"
186952,0,0.594830,"adv, biblioteca cantonale di lugano","in melanges, ed. van den heuvel. paris, gallimard"


0.5948218448218449 <= pubinit_delta <= 0.5948299319727891


,duplicates,pubinit_delta,pubinit_x,pubinit_y
312913,1,1.0,bei a.h. hirsch,bei a.h. hirsch
242597,0,1.0,,
315001,1,1.0,,
78328,0,1.0,,
275815,1,1.0,frenetic films,frenetic films
312765,1,1.0,,
303206,1,1.0,,
285550,1,1.0,staatlicher lehrmittelverlag,staatlicher lehrmittelverlag
63511,0,1.0,,
246155,0,1.0,,


0.9954337899543378 <= pubinit_delta <= 1.0


The similarities of $\texttt{pubinit}$ are affected by empty values. These will be transformed to negative values.

In [63]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### scale

Comparing the similarity metrics of some sample value pairs of attribute $\texttt{scale}$ in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), a Jaccard metrics has been identified to express the best matching behaviour for purely numerical values stored in the attribute.

In [64]:
attribute = 'scale'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [65]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

scale values range [0.         0.17857143 0.21428571 0.57142857 1.        ]


In [66]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,scale_delta,scale_x,scale_y
44605,0,0.0,,100000
142108,0,0.0,,100000
83133,0,0.0,,100000
129657,0,0.0,,100000
19354,0,0.0,,100000
185135,0,0.0,,100000
257265,0,0.0,100000,
23820,0,0.0,,50000
235213,0,0.0,,100000
51381,0,0.0,,100000


0.0 <= scale_delta <= 0.1785714285714286


,duplicates,scale_delta,scale_x,scale_y
227818,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
182750,0,0.571429,50000,100000
227494,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
95851,0,0.571429,50000,100000
69507,0,0.571429,50000,100000
182758,0,0.571429,50000,100000
55255,0,0.571429,50000,100000
182771,0,0.571429,50000,100000
227502,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
182427,0,0.571429,50000,100000


0.2142857142857143 <= scale_delta <= 0.5714285714285714


,duplicates,scale_delta,scale_x,scale_y
227837,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
866,1,0.178571,50000,50 000 8 10 8 35 45 55 46 05
874,1,0.178571,50 000 8 10 8 35 45 55 46 05,50000
227814,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227843,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227839,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227829,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227842,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227821,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227863,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000


0.1785714285714286 <= scale_delta <= 0.2142857142857143


Attribute $\texttt{scale}$ is filled for maps, only. Due to its sparse filling, the similarities of the attribute are affected strongly by empty values. These empty values will be marked with a special negative value.

In [67]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### ttlfull

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{ttlfull}$ has been split up into two new attributes $\texttt{ttlfull}\_\texttt{245}$ and $\texttt{ttlfull}\_\texttt{246}$ which will be compared by the same similarity metrics. A visual analysis of the values stored in the attribute, reveals a string of words, comparable to the strings in attribute $\texttt{person}\_\texttt{245c}$, above. The same similarity metric will be used for both title attributes, therefore.

In [68]:
attribute = 'ttlfull'

columns_metadata_dict['similarity_metrics'][attribute+'_245'] = tedi.Jaro()
columns_metadata_dict['similarity_metrics'][attribute+'_246'] = tedi.Jaro()

tf_values = ['_245', '_246']

for tf in tf_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+tf,
        columns_metadata_dict['similarity_metrics'][attribute+tf],
        columns_metadata_dict)

In [69]:
for tf in tf_values :
    uniques[attribute+tf], uniques_len[attribute+tf] = dpf.determine_similarity_values(
        df_feature_base, attribute+tf)

ttlfull_245 values range [0.         0.24175824 0.24358974 ... 0.99947257 0.99947341 1.        ]
ttlfull_246 values range [0.         0.27407407 0.35555556 ... 0.99945085 0.99945175 1.        ]


In [70]:
tf = '_245'
position = uniques_len[attribute+tf]

dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-position],
    uniques[attribute+tf][uniques_len[attribute+tf]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-3],
    uniques[attribute+tf][uniques_len[attribute+tf]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-4],
    uniques[attribute+tf][uniques_len[attribute+tf]-3], 10)

,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
66476,0,0.0,emma,blick in die welt
11910,0,0.0,emma,blick in die welt
4203,0,0.0,emma,blick in die welt
26234,0,0.0,bildungsforschung und bildungspraxis,emma
4198,0,0.0,emma,blick in die welt
78940,0,0.0,emma,blick in die welt
33582,0,0.0,arts,how do i find the christ?
18740,0,0.0,emma,blick in die welt
13108,0,0.0,emma,blick in die welt
6398,0,0.0,bildungsforschung und bildungspraxis,emma


0.0 <= ttlfull_245_delta <= 0.2417582417582418


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
320301,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...
320267,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...
320317,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...
320322,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...
320264,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...
320249,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...


0.9994725738396625 <= ttlfull_245_delta <= 0.9994734070563455


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
319772,1,0.999462,health informatics - personal health device co...,health informatics - personal health device co...
319719,1,0.999462,health informatics - personal health device co...,health informatics - personal health device co...
319747,1,0.999462,health informatics - personal health device co...,health informatics - personal health device co...
319734,1,0.999462,health informatics - personal health device co...,health informatics - personal health device co...
320317,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...
319782,1,0.999462,health informatics - personal health device co...,health informatics - personal health device co...
319751,1,0.999462,health informatics - personal health device co...,health informatics - personal health device co...
319725,1,0.999462,health informatics - personal health device co...,health informatics - personal health device co...
320322,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...
320264,1,0.999473,health informatics - personal health device co...,health informatics - personal health device co...


0.9994623655913979 <= ttlfull_245_delta <= 0.9994725738396625


Attribute $\texttt{ttlfull}\_\texttt{245}$ is filled for all data rows of Swissbib's raw data as can be seen in chapter [Data Analysis](./1_DataAnalysis.ipynb). For attribute $\texttt{ttlfull}\_\texttt{245}$, the filling is below $10\%$. The data pairs with missing values will be marked with a negative value as has been done for similar cases above.

In [71]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute+'_246', factor)

### volumes

This attribute is described in chapter [Data Analysis](./1_DataAnalysis.ipynb) to hold a kind of contents that resembles the contents of attribute $\texttt{part}$. The same similarity metrics will be used for attribute $\texttt{volumes}$ as for attribute $\texttt{part}$, therefore.

In [72]:
attribute = 'volumes'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
#tedi.Jaro()
#tedi.LCSSeq()
#tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [73]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

volumes values range [0.         0.31318681 0.32222222 0.32905983 0.35128205 0.37301587
 0.37407407 0.38333333 0.38461538 0.3952381  0.40740741 0.41111111
 0.41666667 0.41737892 0.42857143 0.43650794 0.43703704 0.44017094
 0.44166667 0.44200244 0.44230769 0.44761905 0.4537037  0.45555556
 0.45833333 0.46296296 0.46428571 0.46581197 0.46666667 0.47008547
 0.47222222 0.47619048 0.47777778 0.48148148 0.48333333 0.48412698
 0.48611111 0.48888889 0.49007937 0.49145299 0.49206349 0.49365079
 0.4991453  0.5        0.50793651 0.51111111 0.51190476 0.52222222
 0.52380952 0.52564103 0.52777778 0.53333333 0.53653846 0.53703704
 0.53968254 0.54074074 0.54166667 0.54304029 0.54401709 0.54722222
 0.54761905 0.5491453  0.55       0.55128205 0.55555556 0.56031746
 0.56111111 0.56190476 0.56507937 0.56630037 0.56666667 0.56944444
 0.57407407 0.57478632 0.57777778 0.58148148 0.58333333 0.58730159
 0.58862434 0.58888889 0.59444444 0.5952381  0.59722222 0.5982906
 0.6        0.60119048 0.60320513 0.606837

In [74]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,volumes_delta,volumes_x,volumes_y
240477,0,0.0,475,1 170
234478,0,0.0,239,1 406
169692,0,0.0,2,107
186134,0,0.0,549,6
8091,0,0.0,2 109,1
126563,0,0.0,,1 446
25820,0,0.0,,1
208823,0,0.0,,144
100686,0,0.0,252,108
111162,0,0.0,4,1 388


0.0 <= volumes_delta <= 0.3131868131868132


,duplicates,volumes_delta,volumes_x,volumes_y
258760,0,0.916667,1 82,1 2
256981,0,0.916667,1 82,1 2
10889,0,0.916667,1 82,1 2
75477,0,0.933333,1 221,1 21
165068,0,0.916667,1 82,1 2
250040,0,0.933333,1 32,1 132
250460,0,0.916667,1 32,1 2
16434,0,0.933333,1 167,1 16
256983,0,0.916667,1 82,1 2
23932,0,0.916667,1 82,1 2


0.9166666666666666 <= volumes_delta <= 0.9333333333333332


,duplicates,volumes_delta,volumes_x,volumes_y
250458,0,0.916667,1 32,1 2
247197,0,0.916667,1 45,145
7923,0,0.916667,1 82,1 2
28085,0,0.916667,1 82,1 2
7921,0,0.916667,1 82,1 2
256983,0,0.916667,1 82,1 2
258762,0,0.916667,1 82,1 2
258760,0,0.916667,1 82,1 2
23934,0,0.916667,1 82,1 2
250461,0,0.916667,1 32,1 2


0.9047619047619048 <= volumes_delta <= 0.9166666666666666


Attribute $\texttt{volumes}$ holds rows with missing data. The data pairs with missing values will be marked with a special negative value.

In [75]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

## DataFrame with Attributes and Similarity Features

The metric for each attribute of the feature DataFrame has been decided and the similarity features have been calculated. In this last step, the columns of the DataFrame are reordered in order to place the $\_\texttt{delta}$ columns close to their input origins $\_\texttt{x}$ and $\_\texttt{y}$ and some sample records are shown.

In [76]:
# Take _x, _y, and _delta columns together
fb_col_list = df_feature_base.columns.tolist()
fb_col_list.sort()
# Move target column to first place
fb_col_list.insert(0, fb_col_list.pop(fb_col_list.index('duplicates')))
# Reorder DataFrame columns
df_attribute_with_sim_feature = pd.DataFrame(df_feature_base, columns=fb_col_list)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_attribute_with_sim_feature.columns)

class_label = ['uniques', 'duplicate']

for i in class_label:
    display(df_attribute_with_sim_feature[df_attribute_with_sim_feature.duplicates==class_label.index(i)].sample(n=10))
    print(i)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y,coordinate_N_delta,coordinate_N_x,coordinate_N_y,corporate_full_delta,corporate_full_x,corporate_full_y,doi_delta,doi_x,doi_y,edition_delta,edition_x,edition_y,exactDate_delta,exactDate_x,exactDate_y,format_postfix_delta,format_postfix_x,format_postfix_y,format_prefix_delta,format_prefix_x,format_prefix_y,isbn_delta,isbn_x,isbn_y,ismn_delta,ismn_x,ismn_y,musicid_delta,musicid_x,musicid_y,part_delta,part_x,part_y,person_100_delta,person_100_x,person_100_y,person_245c_delta,person_245c_x,person_245c_y,person_700_delta,person_700_x,person_700_y,pubinit_delta,pubinit_x,pubinit_y,scale_delta,scale_x,scale_y,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y,ttlfull_246_delta,ttlfull_246_x,ttlfull_246_y,volumes_delta,volumes_x,volumes_y
227224,0,-0.5,,e0074147,-0.5,,n0460833,-0.5,,schweizeidgenössisches topographisches bureau,-1.0,,,-1.0,,,0.375,1764aaaa,1873uuuu,0.111111,020000,010300,0.0,bk,mp,1.0,[],[],-1.0,,,-1.0,,,-0.500000,,23 1869 23 1869 23,-0.500000,voltaire,,-0.500000,,g. h. dufour direxit ; h. müllhaupt sculpsit,-0.500000,,"dufourguillaume-henri, müllhauptheinrich, manzj.",0.466667,[s.n.],[eidg. topographisches bureau],-0.5,,100000,0.511594,traité sur la tolérance,"domo d'ossola, arona",-0.5,,"[domodossola, arona]",0.777778,191,1
190434,0,-1.0,,,-1.0,,,-1.0,,,-0.5,10.1055/b-002-26639,,-0.5,6,,0.625,2013aaaa,2014uuuu,0.111111,020053,030100,0.0,bk,mu,0.0,"[978-3-13-150826-3, 978-3-13-193106-1]",[978-3-944063-13-3],-1.0,,,-1.0,,,-1.000000,,,0.569865,trappehans-joachim,mozartwolfgang amadeus,0.531807,"hans-joachim trappe, hans-peter schuster","wolfgang amadeus mozart ; idee, text, dramatur...",0.528624,schusterhans-peter,"braunrichard, zamperoniluca, klemmmatti, stern...",-0.500000,,amor verlag,-1.0,,,0.516458,ekg-kurs für isabel,die zauberflöte,-1.0,,,0.722222,1,1 1 12
255256,0,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.500,2017aaaa,2003uuuu,0.111111,010000,020053,1.0,bk,bk,0.0,[],"[978-3-598-31516-9 (print), 978-3-11-097838-4]",-1.0,,,-1.0,,,0.547222,313 2017,36 36,-0.500000,,mortzfeldpeter,0.562639,"reto u. schneider, candida höfer, peter haffne...","mortzfeld, peter; raabe, pa",0.398653,"schneiderreto u., höfercandida, haffnerpeter",raabepaul,-0.500000,,de gruyter saur,-1.0,,,0.622138,"alles wissen dieser welt, warum bibliotheken n...",katalog der graphischen porträts in der herzog...,-1.0,,,0.566667,74,1 441
136804,0,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.500,2005aaaa,uuuuuuuu,0.428571,010100,010200,1.0,mu,mu,1.0,[],[],-0.5,m006204687,,0.5,601,8501,-1.000000,,,1.000000,mozartwolfgang amadeus,mozartwolfgang amadeus,-0.500000,w. a. mozart,,-0.500000,mozartwolfgang amadeus,,-0.500000,bärenreiter,,-1.0,,,0.669415,"die zauberflöte, kv 620","die zauberflöte, oper in zwei akten : bearbeit...",-0.5,,"die zauberflöte, ausgabe für klavier allein",0.733333,1 379,1
230672,0,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.500,1987aaaa,1998uuuu,1.000000,020000,020000,1.0,bk,bk,1.0,[],[],-1.0,,,-1.0,,,-0.500000,1,,-1.000000,,,0.681563,sigrid kessler... [et al.] ; [éd.:] interkanto...,sigrid kessler [u.a.],-1.000000,,,1.000000,staatlicher lehrmittelverlag,staatlicher lehrmittelverlag,-1.0,,,0.798960,"bonne chance!, cours de langue française, deux...","bonne chance!, cours de langue française, nouv...",-1.0,,,0.555556,167,171
119857,0,-1.0,,,-1.0,,,-1.0,,,-0.5,,10.5169/seals-377277,-0.5,3,,0.250,1989aaaa,2005uuuu,0.111111,020000,010053,1.0,bk,bk,1.0,[],[],-1.0,,,-1.0,,,-0.500000,,284 2005,-0.500000,,bührerwalter,0.536501,heidy binder... [et al.] ; [éd.:] interkantona...,[walter bührer],-0.500000,binderheidy,,-0.500000,staatlicher lehrmittelverlag,,-1.0,,,0.535934,"bonne chance!, cours de langue française, prem...",blick in die welt,-1.0,,,-0.500000,413,
98591,0,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.250,2005aaaa,1968uuuu,1.000000,020000,020000,1.0,bk,bk,0.0,[3-8067-5097-1],[],-1.0,,,-1.0,,,-1.000000,,,-0.500000,,mozartwolfgang amadeus,0.567460,luc jacquet ; [übers.: cornelia panzacchi],wolfgang amadeus 

uniques


,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y,coordinate_N_delta,coordinate_N_x,coordinate_N_y,corporate_full_delta,corporate_full_x,corporate_full_y,doi_delta,doi_x,doi_y,edition_delta,edition_x,edition_y,exactDate_delta,exactDate_x,exactDate_y,format_postfix_delta,format_postfix_x,format_postfix_y,format_prefix_delta,format_prefix_x,format_prefix_y,isbn_delta,isbn_x,isbn_y,ismn_delta,ismn_x,ismn_y,musicid_delta,musicid_x,musicid_y,part_delta,part_x,part_y,person_100_delta,person_100_x,person_100_y,person_245c_delta,person_245c_x,person_245c_y,person_700_delta,person_700_x,person_700_y,pubinit_delta,pubinit_x,pubinit_y,scale_delta,scale_x,scale_y,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y,ttlfull_246_delta,ttlfull_246_x,ttlfull_246_y,volumes_delta,volumes_x,volumes_y
308870,1,-1.000,,,-1.0,,,-1.000000,,,-1.0,,,-1.0,,,0.75,2007aaaa,2007uuuu,1.0,020053,020053,1.0,bk,bk,1.0,"[978-3-598-31807-8 (print), 978-3-11-096145-4]","[978-3-598-31807-8 (print), 978-3-11-096145-4]",-1.0,,,-1.0,,,1.0,47 47,47 47,0.980952,mortzfeldpeter,motrzfeldpeter,0.988506,"mortzfeld, peter; raabe, paul","ortzfeld, peter; raabe, paul",0.977778,raabepaul,raabepual,1.000000,de gruyter saur,de gruyter saur,-1.0,,,0.998227,katalog der graphischen porträts in der herzog...,katalog der graphischen porträts in der herzog...,-1.000000,,,1.0,1 392,1 392
271131,1,-1.000,,,-1.0,,,-1.000000,,,-1.0,,,-1.0,,,1.00,19841992,19841992,1.0,030100,030100,1.0,vm,vm,1.0,[],[],-1.0,,,-1.0,,,-1.0,,,-1.000000,,,0.987654,sigrid kessler ... [et al.],sigrid keslser ... [et al.],1.000000,kesslersigrid,kesslersigrid,-1.000000,,,-1.0,,,0.973741,"bonne chance, cours de langue française : exig...","bonne chance, cours de lngue française : exige...",-1.000000,,,-1.0,,
301880,1,-1.000,,,-1.0,,,1.000000,"metropolitan opera (new york)orchestra, metrop...","metropolitan opera (new york)orchestra, metrop...",-1.0,,,-1.0,,,0.75,2000aaaa,2000uuuu,1.0,010000,010000,1.0,vm,vm,1.0,[],[],-1.0,,,1.0,073,073,-1.0,,,1.000000,mozartwolfgang amadeus,mozartwolfgang amadeus,0.986607,w.a. mozart ; libretto: emanuel schikaneder ; ...,ww.a. mozart ; libretto: emanuel schikaneder ;...,0.939213,"schikanederemanuel, coxjohn, levinejames, batt...","schikanederemanuel, coxjohn, levinejaes, battl...",1.000000,"deutsche grammophon, universal music","deutsche grammophon, universal music",-1.0,,,0.935076,"die zauberflöte, oper in zwei aufzügen = the m...","die zauberflöte, oper in zwei aufzügen = the m...",-1.000000,,,1.0,1,1
269455,1,-1.000,,,-1.0,,,-1.000000,,,-1.0,,,-1.0,,,0.75,1955aaaa,1955uuuu,1.0,020000,020000,1.0,bk,bk,1.0,[],[],-1.0,,,-1.0,,,1.0,129,129,1.000000,austenjane,austenjane,0.972222,jane austen,yjane austen,-1.000000,,,-1.000000,,,-1.0,,,0.833333,emma,emmd,-1.000000,,,1.0,432,432
265176,1,-1.000,,,-1.0,,,-1.000000,,,-1.0,,,-1.0,,,0.75,1959aaaa,1959uuuu,1.0,020000,020000,1.0,bk,bk,1.0,[],[],-1.0,,,-1.0,,,1.0,3 7 27,3 7 27,0.958333,voltaire,ovltaire,0.971958,voltaire ; préface de dominique aury,voltaire ; préface deedominique aury,-1.000000,,,0.981481,la guilde du livre,la guilded u livre,-1.0,,,0.985507,traité sur la tolérance,traité sur la tolérace,-1.000000,,,1.0,214,214
296174,1,0.125,e0074147,n0460833,1.0,n0460833,n0460833,0.750000,eidgenössische landestopographie,eidgenössische landestopggraphie,-1.0,,,1.0,1943,1943,0.75,1946aaaa,1946uuuu,1.0,010300,010300,1.0,mp,mp,1.0,[],[],-1.0,,,-1.0,,,1.0,23 23 1946 23,23 23 1946 23,-1.000000,,,1.000000,g. h. dufour direxit ; h. müllhaupt sculpsit,g. h. dufour direxit ; h. müllhaupt sculpsit,1.000000,"dufourguillaume-henri, müllhauptheinrich","dufourguillaume-henri, müllhauptheinrich",1.000000,[eidg. landestopographie],[eidg. landestopographie],1.0,100000,100000,0.967460,"domo d'ossola, arona","domod 'ossola, karona",0.914035,"[domodossola, arona]","[domyodossla, arona]",1.0,1,1
292962,1,-1.000,,,-1.0,,,-1.000000,,,-1.0,,,-1.0,,,0.75,1996aaaa,1996uuuu,1.0,020000,020000,1.0,bk,bk,1.0,[3-596-22191-9],[3-596-22191-9],-1.0,,,-1.0,,,1.0,2191,219

duplicate


## Summary

This chapter covers the central area of feature construction. The features of the feature matrix have been generated for each attribute of Swissbib's raw data, deciding on its similarity metric. With these metric values, the feature base DataFrame has been extended and a new DataFrame with the attribute values of the pairs together with their calculated similarity value have been generated. The similarity values will be the final features for training and performance testing of the models, compare [[JudACaps](./A_References.ipynb#judacaps)].

In [77]:
columns_metadata_dict['similarity_metrics']

{'coordinate_E': LCSStr({'qval': 1, 'external': True}),
 'coordinate_N': LCSStr({'qval': 1, 'external': True}),
 'corporate_full': LCSStr({'qval': 1, 'external': True}),
 'doi': Identity({'qval': 1, 'external': True}),
 'edition': Jaccard({'qval': 1, 'as_set': False, 'external': True}),
 'exactDate': Hamming({'qval': 1, 'test_func': <function Base._ident at 0x115b82268>, 'truncate': False, 'external': True}),
 'format_prefix': Identity({'qval': 1, 'external': True}),
 'format_postfix': Jaccard({'qval': 2, 'as_set': False, 'external': True}),
 'isbn': Identity({'qval': 1, 'external': True}),
 'ismn': Identity({'qval': 1, 'external': True}),
 'musicid': LCSStr({'qval': 1, 'external': True}),
 'part': StrCmp95({'long_strings': False, 'external': True}),
 'person_100': StrCmp95({'long_strings': False, 'external': True}),
 'person_700': StrCmp95({'long_strings': False, 'external': True}),
 'person_245c': Jaro({'qval': 1, 'long_tolerance': False, 'winklerize': False, 'external': True}),
 'pu

The similarity metric decided for each attribute has been added as an additional piece of information to the columns metadata dictionary. The following table gives this summary in a structured form and lists the metric used for each attribute. Attributes with the same font color indicate similar types of values (description column) for better orientation.

| attribute     | subtype | description | similarity metric |
| ------------- |:--------|:------------|:------------------|
|<font color='red'>[coordinate](#coordinate)</font>|<font color='red'>\_E</font>|<font color='red'>Code(9)</font>|<font color='red'>LCSStr</font>|
|               |<font color='red'>\_N</font>|<font color='red'>Code(9)</font>|<font color='red'>LCSStr</font>|
|<font color='blue'>[corporate](#corporate)</font>|<font color='blue'>\_full</font>|<font color='blue'>Name</font>|<font color='blue'>LCSStr</font>|
|<font color='green'>[doi](#doi)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='orange'>[edition](#edition)</font>|         |<font color='orange'>Number</font>|<font color='orange'>Jaccard</font>|
|<font color='black'>[exactDate](#exactDate)</font>|         |<font color='black'>Date</font>|<font color='black'>Hamming</font>|
|<font color='red'>[format](#format)</font>|<font color='red'>\_prefix</font>|<font color='red'>Code(2)</font>|<font color='red'>Identity</font>|
|               |<font color='red'>\_postfix</font>|<font color='red'>Code(6)</font>|<font color='red'>Jaccard (qval=2)</font>|
|<font color='green'>[isbn](#isbn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[ismn](#ismn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[musicid](#musicid)</font>|         |<font color='green'>Identifier</font>|<font color='green'>LCSStr</font>|
|<font color='orange'>[part](#part)</font>|         |<font color='orange'>Number</font>|<font color='orange'>StrCmp95</font>|
|<font color='blue'>[person](#person)</font>|<font color='blue'>\_100</font>|<font color='blue'>Name</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_700</font>|<font color='blue'>Name</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_245c</font>|<font color='blue'>Name</font>|<font color='blue'>Jaro</font>|
|<font color='blue'>[pubinit](#pubinit)</font>|         |<font color='blue'>Name</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[scale](#scale)</font>|         |<font color='orange'>Number</font>|<font color='orange'>Jaccard</font>|
|<font color='blue'>[ttlfull](#ttlfull)</font>|<font color='blue'>\_245</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|               |<font color='blue'>\_246</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[volumes](#volumes)</font>|         |<font color='orange'>Number</font>|<font color='orange'>StrCmp95</font>|

### Full Feature Matrix with Target Vector Handover

To hand over the resulting DataFrame of this chapter, the DataFrame is saved into a pickle file that will be read in the next chapters [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb) as input.

In [78]:
# Store into compressed intermediary file
with bz2.BZ2File(os.path.join(path_goldstandard,
                       'labelled_feature_matrix_full.pkl'), 'w') as df_output_file:
    pk.dump(df_attribute_with_sim_feature, df_output_file)

The full metadata dictionary is to be persisted for handover to subsequent chapters.

In [79]:
# The target is still needed for the feature matrix
columns_metadata_dict['features'].append('duplicates')

for k in columns_metadata_dict.keys():
    print(k, '\n', columns_metadata_dict[k], '\n')

data_analysis_columns 
 ['coordinate_E', 'coordinate_N', 'corporate_full', 'doi', 'edition', 'exactDate', 'format_prefix', 'format_postfix', 'isbn', 'ismn', 'musicid', 'part', 'person_100', 'person_700', 'person_245c', 'pubinit', 'scale', 'ttlfull_245', 'ttlfull_246', 'volumes'] 

columns_to_use 
 ['duplicates', 'coordinate_E_x', 'coordinate_E_y', 'coordinate_N_x', 'coordinate_N_y', 'corporate_full_x', 'corporate_full_y', 'doi_x', 'doi_y', 'edition_x', 'edition_y', 'exactDate_x', 'exactDate_y', 'format_prefix_x', 'format_prefix_y', 'format_postfix_x', 'format_postfix_y', 'isbn_x', 'isbn_y', 'ismn_x', 'ismn_y', 'musicid_x', 'musicid_y', 'part_x', 'part_y', 'person_100_x', 'person_100_y', 'person_700_x', 'person_700_y', 'person_245c_x', 'person_245c_y', 'pubinit_x', 'pubinit_y', 'scale_x', 'scale_y', 'ttlfull_245_x', 'ttlfull_245_y', 'ttlfull_246_x', 'ttlfull_246_y', 'volumes_x', 'volumes_y'] 

similarity_metrics 
 {'coordinate_E': LCSStr({'qval': 1, 'external': True}), 'coordinate_N': L

In [80]:
# Binary intermediary metadata file
with open(os.path.join(path_goldstandard,
                       'columns_metadata.pkl'), 'wb') as dict_output_file:
    pk.dump(columns_metadata_dict, dict_output_file)